In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso,ElasticNet,Ridge
from sklearn.metrics import r2_score, mean_squared_log_error
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sn
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

data = pd.read_csv("../input/hackerearth-carnival-wars-challenge-problem-1/train.csv")
test_data = pd.read_csv("../input/hackerearth-carnival-wars-challenge-problem-1/test.csv")

#data.boxplot(column=[ "Grade", "Demand", "charges_1", "charges_2 (%)"])


updated_data = data
## Derive Column
## Main DATA
new_df = pd.DataFrame()
for i in range(len(updated_data)):
    new_df.loc[i,"Date"] =  str(updated_data.loc[i,"instock_date"])[:10].strip()
updated_data["Date"] = new_df["Date"]

for i in range(len(updated_data)):
    new_df.loc[i,"Year"] =  int(str(updated_data.loc[i,"Date"])[:4].strip())
for i in range(len(updated_data)):
    new_df.loc[i, "Month"] = int(str(updated_data.loc[i, "Date"])[5:7].strip())
for i in range(len(updated_data)):
    new_df.loc[i, "Day"] = int(str(updated_data.loc[i, "Date"])[8:].strip())


updated_data["Year"] = new_df["Year"]
updated_data["Month"] = new_df["Month"]
updated_data["Day"] = new_df["Day"]


encoder = LabelEncoder()
updated_data["Product_Category_1"] = encoder.fit_transform(updated_data["Product_Category"])
updated_data["Loyal_Customer_1"] = encoder.fit_transform(updated_data["Loyalty_customer"])

#print(updated_data[updated_data["charges_1"] > 400][["charges_1", "Selling_Price"]].head(20))
## Test Data

new_df1 = pd.DataFrame()
for i in range(len(test_data)):
    new_df1.loc[i,"Date"] =  str(test_data.loc[i,"instock_date"])[:10].strip()
test_data["Date"] = new_df1["Date"]

for i in range(len(test_data)):
    new_df1.loc[i,"Year"] =  int(str(test_data.loc[i,"Date"])[:4].strip())
for i in range(len(test_data)):
    new_df1.loc[i, "Month"] = int(str(test_data.loc[i, "Date"])[5:7].strip())
for i in range(len(test_data)):
    new_df1.loc[i, "Day"] = int(str(test_data.loc[i, "Date"])[8:].strip())


test_data["Year"] = new_df1["Year"]
test_data["Month"] = new_df1["Month"]
test_data["Day"] = new_df1["Day"]



encoder = LabelEncoder()
test_data["Product_Category_1"] = encoder.fit_transform(test_data["Product_Category"])
test_data["Loyal_Customer_1"] = encoder.fit_transform(test_data["Loyalty_customer"])

## Remove Missing values


updated_data = updated_data.dropna(subset= ['Selling_Price'])

updated_data = updated_data[updated_data["Selling_Price"] > 0]
updated_data = updated_data[updated_data["Maximum_price"] > 0]
updated_data = updated_data[updated_data["Minimum_price"] > 0]
updated_data = updated_data[updated_data["charges_1"] > 0]

updated_data = updated_data.drop(columns=['Customer_name', 'Product_id'], axis=1)
updated_data = updated_data[updated_data["charges_2 (%)"].notna()]
updated_data = updated_data[updated_data["Stall_no"].notna()]
updated_data.loc[updated_data["Discount_avail"].isna(), "Discount_avail"] = 0.14

#print(updated_data)

first_half = updated_data[["Maximum_price","charges_1","Minimum_price"]][updated_data["Minimum_price"] > 10000]
second_half = updated_data[["Maximum_price","charges_1","Minimum_price"]][updated_data["Minimum_price"] < 10000]

first_half["diff"] = first_half["Maximum_price"] - first_half["Minimum_price"]
first_half["diff"].mean()

second_half["diff"] = second_half["Maximum_price"] - second_half["Minimum_price"]
#print("second",second_half["diff"].mean())

for i in updated_data[updated_data["Maximum_price"].isna()].index:
    for j in updated_data[updated_data["Minimum_price"] > 10000].index:
       if j == i:
            updated_data.loc[i, "Maximum_price"] = updated_data.loc[i ,"Minimum_price"] + 5845

    for j in updated_data[updated_data["Minimum_price"] < 10000].index:
        if j == i:
            updated_data.loc[i, "Maximum_price"] = updated_data.loc[i, "Minimum_price"] + 2896


for i in updated_data[updated_data["Minimum_price"].isna()].index:
    for j in updated_data[updated_data["Maximum_price"] > 10000].index:
       if j == i:
            updated_data.loc[i, "Minimum_price"] = updated_data.loc[i ,"Maximum_price"] - 7193

    for j in updated_data[updated_data["Maximum_price"] < 10000].index:
        if j == i:
            updated_data.loc[i, "Minimum_price"] = updated_data.loc[i, "Maximum_price"] - 2296

for i in updated_data[updated_data["charges_1"].isna()].index:
       if updated_data.loc[i,"Product_Category_1"] == 0:
            updated_data.loc[i, "charges_1"] = 298.68
       if updated_data.loc[i,"Product_Category_1"] == 1:
            updated_data.loc[i, "charges_1"] = 248.92
       if updated_data.loc[i,"Product_Category_1"] == 2:
            updated_data.loc[i, "charges_1"] = 135.96
       if updated_data.loc[i,"Product_Category_1"] == 3:
            updated_data.loc[i, "charges_1"] = 374.43
       if updated_data.loc[i,"Product_Category_1"] == 4:
           updated_data.loc[i, "charges_1"] = 250.85
       if updated_data.loc[i,"Product_Category_1"] == 5:
           updated_data.loc[i, "charges_1"] = 274.77
       if updated_data.loc[i,"Product_Category_1"] == 6:
           updated_data.loc[i, "charges_1"] = 124.66
       if updated_data.loc[i,"Product_Category_1"] == 7:
           updated_data.loc[i, "charges_1"] = 150.34
       if updated_data.loc[i,"Product_Category_1"] == 8:
           updated_data.loc[i, "charges_1"] = 247.74
       if updated_data.loc[i,"Product_Category_1"] == 9:
           updated_data.loc[i, "charges_1"] = 556.10

for i in test_data[test_data["Minimum_price"].isna()].index:
    for j in test_data[test_data["Maximum_price"] > 10000].index:
       if j == i:
            test_data.loc[i, "Minimum_price"] = test_data.loc[i ,"Maximum_price"] - 7193

    for j in test_data[test_data["Maximum_price"] < 10000].index:
        if j == i:
            test_data.loc[i, "Minimum_price"] = test_data.loc[i, "Maximum_price"] - 2296


for i in test_data[test_data["charges_1"].isna()].index:
       if test_data.loc[i,"Product_Category_1"] == 0:
            test_data.loc[i, "charges_1"] = 298.68
       if test_data.loc[i,"Product_Category_1"] == 1:
            test_data.loc[i, "charges_1"] = 248.92
       if test_data.loc[i,"Product_Category_1"] == 2:
            test_data.loc[i, "charges_1"] = 135.96
       if test_data.loc[i,"Product_Category_1"] == 3:
            test_data.loc[i, "charges_1"] = 374.43
       if test_data.loc[i,"Product_Category_1"] == 4:
           test_data.loc[i, "charges_1"] = 250.85
       if test_data.loc[i,"Product_Category_1"] == 5:
           test_data.loc[i, "charges_1"] = 274.77
       if test_data.loc[i,"Product_Category_1"] == 6:
           test_data.loc[i, "charges_1"] = 124.66
       if test_data.loc[i,"Product_Category_1"] == 7:
           test_data.loc[i, "charges_1"] = 150.34
       if test_data.loc[i,"Product_Category_1"] == 8:
           test_data.loc[i, "charges_1"] = 247.74
       if test_data.loc[i,"Product_Category_1"] == 9:
           test_data.loc[i, "charges_1"] = 556.10


updated_data = updated_data[updated_data["Selling_Price"] < updated_data["Maximum_price"]]
#updated_data = updated_data[updated_data["Selling_Price"] > updated_data["Minimum_price"]]


updated_data = updated_data[updated_data["Maximum_price"] < 30000]
updated_data.to_csv("testingdata.csv")
#"Grade","Day","Product_Category_1",,"Market_Category","Demand","Year"
X_train,X_test,Y_train,Y_test = train_test_split(updated_data[["Maximum_price","Minimum_price","Discount_avail","Loyal_Customer_1"]],updated_data["Selling_Price"], test_size=.25, random_state=899)
#min_samples_split=50,max_leaf_nodes=50
#clf = RandomForestRegressor(max_depth=10,n_estimators=100,max_features=3)

#clf = KNeighborsRegressor(n_neighbors=n)


# Create a based model
clf = GradientBoostingRegressor(max_depth=8,max_features=3,n_estimators=100,min_samples_split=5,min_impurity_decrease=0.05,max_leaf_nodes=300)
#41


#{'criterion': 'mae', 'loss': 'ls'}

#,"Grade","Demand","Year","Market_Category","Day",,"Product_Category_1"
#model = clf.fit(updated_data[["Maximum_price","Minimum_price","Discount_avail","charges_1","Loyal_Customer_1"]],updated_data["Selling_Price"])
model = clf.fit(X_train,Y_train)
#"Grade","Demand","Year","Market_Category","Day","Product_Category_1"
#pred = model.predict(test_data[["Maximum_price","Minimum_price","Discount_avail","charges_1","Loyal_Customer_1"]])
pred = model.predict(X_test)

from sklearn import tree
#print(clf.decision_path(X_train))
#print(clf.criterion)
#print(clf.feature_importances_)
print('R2 score :',r2_score(pred, Y_test))
print('MSLE score',max(0,100 - mean_squared_log_error(Y_test,pred)))
#print(pred)

#print(len(data["charges_1"].unique()))
#sn.pairplot(updated_data[["Discount_avail","Maximum_price","Selling_Price","Year","Month","Grade","Demand","charges_1"]])
#plt.show()


